In [ ]:
import pandas as pd
import numpy as np

def clean_survey_data(file_path):
    # Load dataset
    try:
        df = pd.read_csv(file_path)
        print("Data loaded successfully.")
        
        # Remove the header row that contains question text
        df = df.iloc[1:]
        print("Header row removed.")
        
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return
    except Exception as e:
        print(f"Error loading data: {e}")
        return
    
    # Remove metadata columns
    df = df.iloc[:, 16:]
    print(df.columns)
    
    # Q1: Ward (One-Hot Encoding)
    ward_dummies = pd.get_dummies(df["Q1"], prefix="Ward")
    ward_columns = ["Ward_1", "Ward_3", "Ward_5", "Ward_6", "Ward_7", "Ward_8", 
                   "Ward_9", "Ward_11", "Ward_12", "Ward_14", "Ward_15"]
    
    # Add missing ward columns with zeros
    for col in ward_columns:
        if col not in ward_dummies.columns:
            ward_dummies[col] = 0
            
    # Keep only specified ward columns
    ward_dummies = ward_dummies[ward_columns]
    df = pd.concat([df, ward_dummies], axis=1)
    df.drop("Q1", axis=1, inplace=True)
    print("Ward encoding complete.")

    # Q2: Age (Ordinal Encoding)
    age_mapping = {
        "18-24": 1, 
        "25-29": 2, 
        "30-39": 3, 
        "40-49": 4, 
        "50-64": 5, 
        "65 or older": 6
    }
    df["Age"] = df["Q2"].map(age_mapping).fillna(0)
    df.drop("Q2", axis=1, inplace=True)
    print("Age encoding complete.")

    # Q3: Gender ID (One-Hot Encoding)
    genederdf = pd.get_dummies(df["Q3"], prefix="Gender")
    df.drop("Q3", axis=1, inplace=True)
    df = pd.concat([df, genederdf], axis=1)
    print("Gender encoding complete.")

    # Q4: Marital Status (Binary Encoding)
    print(df.columns)
    df["Marital_Status"] = df["Q4"].map({"Married": 1, "Not married": 0}).fillna(0)
    df.drop("Q4", axis=1, inplace=True)
    print("Marital Status encoding complete.")

    # Q5: Ethnicity (Multi-Hot Encoding)
    ethnicity_cols = [
        "White", "Black or African American", "Hispanic or Latino/a", 
        "Asian", "American Indian or Alaska Native", 
        "Native Hawaiian or Pacific Islander", "Other"
    ]
    for col in ethnicity_cols:
        df[f"Ethnicity_{col}"] = df["Q5"].str.contains(col, na=False).astype(int)
    df.drop("Q5", axis=1, inplace=True)
    print("Ethnicity encoding complete.")

    # Q6: Religion (One-Hot Encoding)
    religiondf = pd.get_dummies(df["Q6"], prefix="Religion")
    df.drop("Q6", axis=1, inplace=True)
    df = pd.concat([df, religiondf], axis=1)
    print("Religion encoding complete.")

    # Q7: Education Level (Ordinal Encoding)
    education_mapping = {
        "Less than high school": 1,
        "High school graduate / GED": 2,
        "Some college": 3,
        "2 year degree": 4,
        "4 year degree": 5,
        "Professional degree": 6,
        "Doctorate": 7
    }
    df["Education"] = df["Q7"].map(education_mapping).fillna(0)
    df.drop("Q7", axis=1, inplace=True)
    print("Education encoding complete.")

    # Q8: Household Income (Ordinal Encoding)
    income_mapping = {
        "Less than $30,000": 1,
        "$30,000-$49,999": 2,
        "$50,000-$100,000": 3,
        "$100,000-$199,999": 4,
        "$200,000 or more": 5
    }
    df["Income"] = df["Q8"].map(income_mapping).fillna(0)
    df.drop("Q8", axis=1, inplace=True)
    print("Income encoding complete.")

    # Q9: Political Affiliation (One-Hot Encoding)
    politicaldf = pd.get_dummies(df["Q9"], prefix="Political")
    df.drop("Q9", axis=1, inplace=True)
    df = pd.concat([df, politicaldf], axis=1)
    print("Political affiliation encoding complete.")

    # Q10-11: Direction Opinions (Ordinal Encoding)
    direction_mapping = {
        "Definitely Wrong": 1,
        "Somewhat Wrong": 2,
        "Unsure": 3,
        "Somewhat Right": 4,
        "Definitely Right": 5
    }
    df["US_Direction"] = df["Q10"].map(direction_mapping).fillna(0)
    df["RI_Direction"] = df["Q11"].map(direction_mapping).fillna(0)
    df.drop(["Q10", "Q11"], axis=1, inplace=True)
    print("Direction opinions encoding complete.")

    # Q12: Presidential Vote (One-Hot Encoding)
    presidentialdf = pd.get_dummies(df["Q12"], prefix="Vote")
    df.drop("Q12", axis=1, inplace=True)
    df = pd.concat([df, presidentialdf], axis=1)
    print("Presidential vote encoding complete.")

    # Q13-16: Confidence Measures (Ordinal Encoding)
    excitement_mapping = {
        "Not at all Excited": 1,
        "Not Very Excited": 2,
        "Somewhat Excited": 3,
        "Very Excited": 4,
        "Extremely Excited": 5
    }
    
    confidence_mapping = {
        "Not at All Confident": 1,
        "Not Very Confident": 2,
        "Unsure": 3,
        "Somewhat Confident": 4,
        "Very Confident": 5
    }
    
    df["Vote_Excitement"] = df["Q13"].map(excitement_mapping).fillna(0)
    df["Politician_Confidence"] = df["Q14"].map(confidence_mapping).fillna(0)
    df["Officials_Confidence"] = df["Q15"].map(confidence_mapping).fillna(0)
    df["Results_Confidence"] = df["Q16"].map(confidence_mapping).fillna(0)
    df.drop(["Q13", "Q14", "Q15", "Q16"], axis=1, inplace=True)
    print("Confidence measures encoding complete.")

    # Q17-18: Voting Information and Reasons (Multi-Hot Encoding)
    info_sources = [
        "Rhode Island Secretary of State's Website",
        "Social Media",
        "Friends/Family",
        "Religious/Community Organizations",
        "The Voter Information Handbook sent to my house",
        "211 Voter Hotline"
    ]
    
    voting_reasons = [
        "Today was the best option for my schedule",
        "I wanted to wait to learn more about the candidates",
        "I did not feel like I had all the information necessary to vote early or by mail",
        "I prefer voting on election day",
        "I forgot to vote early or by mail",
        "I do not trust mail-in voting"
    ]
    
    for source in info_sources:
        df[f"Info_{source}"] = df["Q17"].str.contains(source, na=False).astype(int)
    
    for reason in voting_reasons:
        df[f"Reason_{reason}"] = df["Q18"].str.contains(reason, na=False).astype(int)
    
    df.drop(["Q17", "Q18"], axis=1, inplace=True)
    print("Voting information and reasons encoding complete.")

    # Q19-20: Support Measures (Ordinal Encoding)
    support_mapping = {
        "Definitely Not": 1,
        "Probably Not": 2,
        "I would need more information to make a decision": 3,
        "Probably": 4,
        "Definitely": 5
    }
    
    df["Registration_Support"] = df["Q19"].map(support_mapping).fillna(0)
    df["RCV_Support"] = df["Q20"].map(support_mapping).fillna(0)
    df.drop(["Q19", "Q20"], axis=1, inplace=True)
    print("Support measures encoding complete.")

    # Save cleaned data
    output_path = file_path.replace("survey_data.csv", "cleaned_survey_data.csv")
    df.to_csv(output_path, index=False)
    print(f"Data cleaning complete. Saved as '{output_path}'")

# Example usage
clean_survey_data(r"C:\Users\headl\Documents\EVC\RI-Voting-Models\raw_data\survey_data.csv")

Data loaded successfully.
Header row removed.
Index(['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Unnamed: 22', 'Q7', 'Q8', 'Q9',
       'Q10', 'Q11', 'Q12', 'Q13', 'Q14', 'Q15', 'Q16', 'Q17', 'Q18', 'Q19',
       'Q20'],
      dtype='object')
Ward encoding complete.
Age encoding complete.
Gender encoding complete.
Index(['Q4', 'Q5', 'Q6', 'Unnamed: 22', 'Q7', 'Q8', 'Q9', 'Q10', 'Q11', 'Q12',
       'Q13', 'Q14', 'Q15', 'Q16', 'Q17', 'Q18', 'Q19', 'Q20', 'Ward_1',
       'Ward_3', 'Ward_5', 'Ward_6', 'Ward_7', 'Ward_8', 'Ward_9', 'Ward_11',
       'Ward_12', 'Ward_14', 'Ward_15', 'Age', 'Gender_Female', 'Gender_Male',
       'Gender_Non-binary / Other', 'Gender_Prefer not to say'],
      dtype='object')
Marital Status encoding complete.
Ethnicity encoding complete.
Religion encoding complete.
Education encoding complete.
Income encoding complete.
Political affiliation encoding complete.
Direction opinions encoding complete.
Presidential vote encoding complete.
Confidence measures encoding comp

In [10]:
cleaned_data = pd.read_csv(r"C:\Users\headl\Documents\EVC\RI-Voting-Models\raw_data\cleaned_survey_data.csv")
cleaned_data.head()

,Unnamed: 22,Ward_1,Ward_3,Ward_5,Ward_6,Ward_7,Ward_8,Ward_9,Ward_11,Ward_12,...,Info_The Voter Information Handbook sent to my house,Info_211 Voter Hotline,Reason_Today was the best option for my schedule,Reason_I wanted to wait to learn more about the candidates,Reason_I did not feel like I had all the information necessary to vote early or by mail,Reason_I prefer voting on election day,Reason_I forgot to vote early or by mail,Reason_I do not trust mail-in voting,Registration_Support,RCV_Support
0,NaN,False,False,True,False,False,False,False,False,False,...,0,0,1,0,0,0,0,0,5.0,3.0
1,NaN,False,False,False,False,False,False,False,False,False,...,0,0,0,0,0,0,0,0,5.0,5.0
2,NaN,False,False,False,False,False,False,True,False,False,...,0,0,1,1,1,0,0,0,5.0,5.0
3,NaN,True,False,False,False,False,False,False,False,False,...,0,0,0,0,0,1,1,0,5.0,2.0
4,NaN,False,True,False,False,False,False,False,False,False,...,0,0,0,0,0,1,1,0,5.0,4.0
